In [ ]:
from demo import get_train_data
dataset=get_train_data()

In [ ]:
from transformers import BeitFeatureExtractor, BeitForImageClassification
feature_extractor = BeitFeatureExtractor.from_pretrained('microsoft/beit-base-patch16-224')
model = BeitForImageClassification.from_pretrained('microsoft/beit-base-patch16-224')

In [ ]:
def embed(images):
    inputs = feature_extractor(images=images, return_tensors="pt")
    outputs = model(**inputs,output_hidden_states= True)
    last_hidden=outputs.hidden_states[-1]
    pooler=model.base_model.pooler
    final_emb=pooler(last_hidden).detach().numpy()
    return final_emb    

In [ ]:
ds_with_embeddings = dataset.map(lambda x: {"beit_embeddings":embed(x["image"])},batched=True,batch_size=20)

In [ ]:
ds_with_embeddings.add_faiss_index(column='beit_embeddings')
ds_with_embeddings.save_faiss_index('beit_embeddings', 'beit_index.faiss')
# dataset.load_faiss_index('beit_embeddings', 'beit_index.faiss')

In [ ]:
from IPython.display import display

In [ ]:
query_image=dataset[0]["image"]
scores, retrieved_examples=ds_with_embeddings.get_nearest_examples('beit_embeddings', embed(query_image), k=5)

In [ ]:
for x in retrieved_examples["image"]:
    display(x)